# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

In [1]:
import numpy as np
import pandas as pd
#Import dataframe flights
flights_test = pd.read_csv("flights_test.csv",low_memory=False).drop('Unnamed: 0',axis=1)
flights = pd.read_csv("flights.csv",low_memory=False).drop('Unnamed: 0',axis=1)[list(flights_test.columns)+['arr_delay','taxi_in','taxi_out']]

In [2]:
flights=flights.dropna()


## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

In [3]:
def time_machine(df):
    df['months']=df.fl_date.str[5:7].astype(int)
    df['weekday']=pd.to_datetime(df.fl_date).dt.dayofweek
time_machine(flights)
time_machine(flights_test)

In [4]:
def traffic(df):
    origin_traffic=df.groupby(['fl_date','origin']).sum()[['flights']].reset_index().rename({'flights': 'origin_traffic'}, axis='columns')
    dest_traffic=df.groupby(['fl_date','dest']).sum()[['flights']].reset_index().rename({'flights': 'dest_traffic'}, axis='columns')
    df = pd.merge(df, origin_traffic, on=['fl_date','origin'])
    df=pd.merge(df, dest_traffic, on=['fl_date','dest'])
    return df

In [5]:
flights_test=traffic(flights_test)
flights=traffic(flights)

In [6]:
def time_to_mins(df):
    df['crs_dep_time(mins)']=((df.crs_dep_time//100)*60)+df.crs_dep_time-(df.crs_dep_time//100)*100
    df['crs_arr_time(mins)']=((df.crs_arr_time//100)*60)+df.crs_arr_time-(df.crs_arr_time//100)*100
    

In [7]:
time_to_mins(flights)

In [8]:
time_to_mins(flights_test)

In [9]:
def taxi_Med(df):
    origin_taxi=flights.groupby('origin').taxi_out.median().to_frame().reset_index().rename({'taxi_out': 'origin_taxi'},axis='columns')
    dest_taxi=flights.groupby('dest').taxi_in.median().to_frame().reset_index().rename({'taxi_in': 'dest_taxi'},axis='columns')
    df=pd.merge(df,origin_taxi , on='origin')  
    df=pd.merge(df, dest_taxi, on='dest')
    return df


In [10]:
flights_test=taxi_Med(flights_test)
flights=taxi_Med(flights)

In [11]:
def one_hot(df):
    df = df.join(pd.get_dummies(df.mkt_carrier))
    return df

In [17]:
flights_test=one_hot(flights_test)

In [18]:
flights=one_hot(flights)

In [20]:
features=['mkt_carrier_fl_num','crs_dep_time(mins)','crs_arr_time(mins)','crs_elapsed_time','distance','months','weekday','origin_traffic','dest_traffic','origin_taxi','dest_taxi']+list(flights_test.mkt_unique_carrier.unique())

In [21]:
y=flights.arr_delay
fl=flights[features]
fl_t=flights_test[features]


In [25]:
y

0            4.0
1            7.0
2           -2.0
3            3.0
4          -15.0
            ... 
15609660     5.0
15609661    -5.0
15609662   -10.0
15609663    10.0
15609664     5.0
Name: arr_delay, Length: 15609665, dtype: float64

In [26]:
fl

,mkt_carrier_fl_num,crs_dep_time(mins),crs_arr_time(mins),crs_elapsed_time,distance,months,weekday,origin_traffic,dest_traffic,origin_taxi,...,WN,UA,AS,AA,DL,B6,F9,HA,NK,G4
0,502,490,685,135.0,853.0,10,0,277.0,789.0,15.0,...,1,0,0,0,0,0,0,0,0,0
1,1009,935,1130,135.0,853.0,10,0,277.0,789.0,15.0,...,1,0,0,0,0,0,0,0,0,0
2,1977,805,1000,135.0,853.0,10,0,277.0,789.0,15.0,...,1,0,0,0,0,0,0,0,0,0
3,2132,630,825,135.0,853.0,10,0,277.0,789.0,15.0,...,1,0,0,0,0,0,0,0,0,0
4,2299,971,1165,134.0,853.0,10,0,277.0,789.0,15.0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15609660,6391,810,845,35.0,73.0,3,4,5.0,1.0,10.0,...,0,1,0,0,0,0,0,0,0,0
15609661,6391,810,845,35.0,73.0,3,6,5.0,1.0,10.0,...,0,1,0,0,0,0,0,0,0,0
15609662,6391,810,845,35.0,73.0,3,0,5.0,1.0,10.0,...,0,1,0,0,0,0,0,0,0,0
15609663,6391,810,845,35.0,73.0,3,3,5.0,1.0,10.0,...,0,1,0,0,0,0,0,0,0,0


In [27]:
fl_t

,mkt_carrier_fl_num,crs_dep_time(mins),crs_arr_time(mins),crs_elapsed_time,distance,months,weekday,origin_traffic,dest_traffic,origin_taxi,...,WN,UA,AS,AA,DL,B6,F9,HA,NK,G4
0,5888,1090,1185,95,363,1,2,66,455,10.0,...,1,0,0,0,0,0,0,0,0,0
1,6276,710,800,90,363,1,2,66,455,10.0,...,1,0,0,0,0,0,0,0,0,0
2,5286,959,1058,99,363,1,2,66,455,10.0,...,0,1,0,0,0,0,0,0,0,0
3,5366,1150,1249,99,363,1,2,66,455,10.0,...,0,1,0,0,0,0,0,0,0,0
4,5416,671,770,99,363,1,2,66,455,10.0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
660413,2158,1049,1114,125,706,1,4,30,1,13.0,...,0,0,0,0,0,0,0,0,0,1
660414,2158,1049,1114,125,706,1,0,30,1,13.0,...,0,0,0,0,0,0,0,0,0,1
660415,2158,1049,1114,125,706,1,4,30,1,13.0,...,0,0,0,0,0,0,0,0,0,1
660416,2158,1049,1114,125,706,1,0,30,1,13.0,...,0,0,0,0,0,0,0,0,0,1


## Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.